In [1]:
import os, sys
sys.path.append('.')

from dataclasses import dataclass
from pathlib import Path
from torch.utils.data import random_split

import draccus
import torch
import torch.distributed as dist
import tqdm
import wandb
from accelerate import PartialState
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.optim import AdamW
from torch.utils.data import DataLoader
from transformers import AutoModelForVision2Seq, AutoProcessor, BitsAndBytesConfig
from transformers.modeling_outputs import CausalLMOutputWithPast

from vla.base_prompter import PurePromptBuilder
from vla.utils import PaddedCollatorForPosePrediction, runningLoss, SamplerForPosePrediction
from vla.action_tokenizer import RLbenchPoseTokenizer
from vla.dataset import RLbenchCotDataset
import numpy as np
import torch.nn.functional as F


2024-08-06 06:18:20.221255: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-06 06:18:20.221307: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-06 06:18:20.229353: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-06 06:18:20.237053: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-06 06:18:21.460153: W tensorflow/compiler/tf2

In [2]:
@dataclass
class FinetuneConfig:
    # fmt: off
    vla_path: str = "/media/lawrence/Work/checkpoints/ecot-openvla-7b-bridge"   # Path to OpenVLA model 
    vla_path_q: str = "/media/lawrence/Work/checkpoints/ecot-openvla-7b-bridge-4b"   # Path to OpenVLA model

    experiment_name: str = "weighted_loss_cot_"
    dataset_name: str = "pick_described_object_replay"                                # Name of fine-tuning dataset (e.g., `droid_wipe`)
    # data_path: Path = Path(f"./datasets/{dataset_name}/data.pt")
    train_data_path: Path = Path(f"./datasets/{dataset_name}/data.pt")
    # test_data_path: Path = Path(f"./datasets/{dataset_name}/test_data.pt")
    item_num = 5
    stage_num = 2
    run_root_dir: Path = Path("./runs")                               # Path to directory to store logs & checkpoints
    adapter_dir: Path = Path("./adapter-tmp")                     # Temporary directory for LoRA weights before fusing

    # Fine-tuning Parameters
    seed: int = 42                                                  # Random seed
    episode: int = 2
    batch_size: int = 2#16                                            # Fine-tuning batch size
    test_limit_length: int = 30
    save_steps: int = 20#5000                                          # Interval for checkpoint saving
    learning_rate: float = 5e-5                                     # Fine-tuning learning rate
    weight_decay: float = 0.01                                      # Fine-tuning weight decay
    grad_accumulation_steps: int = 10                                # Gradient accumulation steps

    # LoRA Arguments
    use_lora: bool = True                                           # Whether to use LoRA fine-tuning
    lora_rank: int = 16#32                                             # Rank of LoRA weight matrix
    lora_dropout: float = 0.0                                       # Dropout applied to LoRA weights
    use_quantization: bool = True                                  # Whether to 4-bit quantize VLA for LoRA fine-tuning
                                                                    #   => CAUTION: Reduces memory but hurts performance
    dataset_statistics: tuple = (np.array([-0.2, -0.35,  0.75199986, -np.pi/2, -np.pi/2, -np.pi/2,  0. ]), np.array([0.5, 0.35, 1.3, np.pi/2, 0, np.pi/2, 1.])) # Min-Max normalization statistics

    # Tracking Parameters
    wandb_project: str = "vla-rl"                                  # Name of W&B project to log to (use default!)
    wandb_entity: str = "lawrence-rs-lin-university-of-toronto"                           # Name of entity to log under

    # fmt: on
cfg = FinetuneConfig()

In [3]:
print(f"Fine-tuning OpenVLA Model `{cfg.vla_path}` on `{cfg.dataset_name}`")

# [Validate] Ensure GPU Available & Set Device / Distributed Context
assert torch.cuda.is_available(), "Fine-tuning assumes at least one GPU is available!"
distributed_state = PartialState()
torch.cuda.set_device(device_id := distributed_state.local_process_index)
torch.cuda.empty_cache()

# Configure Unique Experiment ID & Log Directory
exp_id = (
    f"{cfg.vla_path.split('/')[-1]}+{cfg.dataset_name}"
    f"+b{cfg.batch_size * cfg.grad_accumulation_steps}"
    f"+lr-{cfg.learning_rate}"
)
if cfg.use_lora:
    exp_id += f"+lora-r{cfg.lora_rank}+dropout-{cfg.lora_dropout}"
if cfg.use_quantization:
    exp_id += "+q-4bit"

# Start =>> Build Directories
run_dir, adapter_dir = cfg.run_root_dir / exp_id, cfg.adapter_dir / exp_id
os.makedirs(run_dir, exist_ok=True)

# Quantization Config =>> only if LoRA fine-tuning
quantization_config = None
if cfg.use_quantization:
    assert cfg.use_lora, "Quantized training only supported for LoRA fine-tuning!"
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16, bnb_4bit_quant_type="nf4", #llm_int8_skip_modules = ['projector'],
    )

# Load OpenVLA Processor and Model using HF AutoClasses
processor = AutoProcessor.from_pretrained(cfg.vla_path, trust_remote_code=True)
add_tokens = ['<g>', '</g>'] + [f'<item_{i}>' for i in np.arange(cfg.item_num)] + ['<o>', '</o>', '<t>', '</t>'] + [f'<stage_{i}>' for i in np.arange(cfg.stage_num)] + ['<a>', '</a>', '<q>', '<cot>']
processor.tokenizer.add_tokens(add_tokens)
vla = AutoModelForVision2Seq.from_pretrained(
    cfg.vla_path,
    torch_dtype=torch.bfloat16,
    attn_implementation="sdpa",
    quantization_config=quantization_config,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
    device_map = "cuda"
)


Fine-tuning OpenVLA Model `/media/lawrence/Work/checkpoints/ecot-openvla-7b-bridge` on `pick_described_object_replay`


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

OpenVLAForActionPrediction(
  (vision_backbone): PrismaticVisionBackbone(
    (featurizer): VisionTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14))
        (norm): Identity()
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (patch_drop): Identity()
      (norm_pre): Identity()
      (blocks): Sequential(
        (0): Block(
          (norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
          (attn): Attention(
            (qkv): Linear4bit(in_features=1152, out_features=3456, bias=True)
            (q_norm): Identity()
            (k_norm): Identity()
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear4bit(in_features=1152, out_features=1152, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
          )
          (ls1): Identity()
          (drop_path1): Identity()
          (norm2): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
         

In [4]:
# Device Placement =>> note that BitsAndBytes automatically handles for quantized training
if cfg.use_quantization:
    vla = prepare_model_for_kbit_training(vla)
else:
    vla = vla.to(device_id)

# [LoRA] Wrap Model w/ PEFT `LoraConfig` =>> by default we set `target_modules=all-linear`
if cfg.use_lora:
    lora_config = LoraConfig(
        r=cfg.lora_rank,
        lora_alpha=min(cfg.lora_rank, 16),
        lora_dropout=cfg.lora_dropout,
        target_modules="all-linear",
        init_lora_weights="gaussian",
    )
    vla = get_peft_model(vla, lora_config)
    vla.print_trainable_parameters()

# # Wrap VLA in PyTorch DDP Wrapper for Multi-GPU Training
# vla = DDP(vla, device_ids=[device_id], find_unused_parameters=True, gradient_as_bucket_view=True)

# Create Optimizer =>> note that we default to a simple constant learning rate!
trainable_params = [param for param in vla.parameters() if param.requires_grad]
optimizer = AdamW(trainable_params, lr=cfg.learning_rate)


trainable params: 48,801,984 || all params: 7,236,926,592 || trainable%: 0.6743


In [5]:
# Create Action Tokenizer
action_tokenizer = RLbenchPoseTokenizer(processor.tokenizer, cfg.dataset_statistics)

trainset = RLbenchCotDataset(
    cfg.train_data_path,
    action_tokenizer,
    processor.tokenizer,
    image_transform=processor.image_processor.apply_transform,
)

train_size = int(0.8 * len(trainset))
val_size = len(trainset) - train_size
train_dataset, val_dataset = random_split(trainset, [train_size, val_size])

# Create Collator and DataLoader
collator = PaddedCollatorForPosePrediction(
    processor.tokenizer.model_max_length, processor.tokenizer.pad_token_id, padding_side="right"
)

# train_sampler = SamplerForPosePrediction(np.array(trainset.data['stages']),group1_ratio=0.3)
# test_sampler = SamplerForPosePrediction(np.array(testset.data['stages']),group1_ratio=0.3)

train_dataloader = DataLoader(
    train_dataset,
    batch_size=cfg.batch_size,
    shuffle=True,
    # sampler=train_sampler,
    collate_fn=collator,
    num_workers=1,  # Important =>> Set to 0 if using RLDS; TFDS rolls its own parallelism!
)

test_dataloader = DataLoader(
    val_dataset,
    batch_size=cfg.batch_size,
    shuffle=False,
    sampler=None,
    collate_fn=collator,
    num_workers=1,  # Important =>> Set to 0 if using RLDS; TFDS rolls its own parallelism!
)

In [14]:
next(iter(train_dataloader))

{'pixel_values': tensor([[[[-0.9608, -0.9765, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
           [-0.9608, -0.9843, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
           [-0.9765, -0.9843, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
           ...,
           [ 0.5765,  0.5451,  0.5529,  ...,  0.4980,  0.5686,  0.6235],
           [ 0.5765,  0.5686,  0.5922,  ...,  0.5529,  0.5373,  0.5843],
           [ 0.6157,  0.6157,  0.6392,  ...,  0.6157,  0.5451,  0.5843]],
 
          [[-0.5608, -0.5765, -0.6000,  ..., -0.5843, -0.5686, -0.5686],
           [-0.5608, -0.5843, -0.6000,  ..., -0.5843, -0.5686, -0.5686],
           [-0.5765, -0.5843, -0.6000,  ..., -0.5843, -0.5686, -0.5686],
           ...,
           [ 0.2392,  0.2078,  0.2157,  ...,  0.2157,  0.2863,  0.3412],
           [ 0.2392,  0.2314,  0.2549,  ...,  0.2706,  0.2549,  0.3020],
           [ 0.2784,  0.2784,  0.3020,  ...,  0.3333,  0.2627,  0.3020]],
 
          [[-0.5294, -0.5451, -0.5686,  ..., -0.5529, -0.5451, -0.5451

In [7]:
train_running_loss = runningLoss()

In [14]:
vla.train()
vla.gradient_checkpointing_enable()
optimizer.zero_grad()

step_idx, batch = next(enumerate(train_dataloader))

with torch.autocast("cuda", dtype=torch.bfloat16):
    vla.train()
    output: CausalLMOutputWithPast = vla(
        input_ids=batch["input_ids"].to(device_id),
        attention_mask=batch["attention_mask"].to(device_id),
        pixel_values=batch["pixel_values"].to(torch.bfloat16).to(device_id),
        labels=batch["labels"],
        use_cache=False
    )
    


In [16]:
batch['labels']

tensor([[ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100, 32004,
         29892, 32008,  -100,  -100,  -100, 32009, 29892, 32010, 31398, 31501,
         31643, 32011, 29892, 32013, 29892, 32014,  -100,  -100,  -100,  -100,
          -100,  -100, 31999, 32015, 29892, 32016,  -100,     2],
        [ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100, 

In [10]:
processor.tokenizer.batch_decode(batch['input_ids'])

['<s> In: What should be the next key pose of the gripper to put the spam can in the basket? The current gripper pose is<g>港－局안ḳὀ弘</g>.\n Out:<cot><item_2>,<o>任세漢</o>,<t>交ペམ</t>,<stage_1>,<a>交ペ电면ḳ收给</a>,<q><PAD></s><PAD><PAD>',
 '<s> In: What should be the next key pose of the gripper to put the chocolate gelatin in the basket? The current gripper pose is<g>没민奈ṯ∇昭给</g>.\n Out:<cot><item_0>,<o>计支関</o>,<t>交ペམ</t>,<stage_0>,<a>序拉共면ḳ收弘</a>,<q><PAD></s>']

In [10]:
output_start_idx = vla.vision_backbone.featurizer.patch_embed.num_patches
loss_dict = action_tokenizer.get_loss(output, batch, output_start_idx)
normalized_loss_dict = train_running_loss.update(loss_dict = loss_dict)


ValueError: Expected input batch_size (0) to match target batch_size (2).

In [ ]:
loss_dict

{'nll_loss': tensor(19.6134, device='cuda:0', grad_fn=<NllLossBackward0>),
 'item_loss': tensor(2.9242, device='cuda:0', grad_fn=<NllLossBackward0>),
 'object_position_loss': tensor(0.0366, device='cuda:0', grad_fn=<MseLossBackward0>),
 'target_position_loss': tensor(0.0808, device='cuda:0', grad_fn=<MseLossBackward0>),
 'stage_loss': tensor(2.2828, device='cuda:0', grad_fn=<NllLossBackward0>),
 'action_position_loss': tensor(0.0714, device='cuda:0', grad_fn=<MseLossBackward0>),
 'action_orientation_loss': tensor(0.3213, device='cuda:0', grad_fn=<DivBackward0>),
 'action_open_loss': tensor(3.8399, device='cuda:0', grad_fn=<NllLossBackward0>)}

In [ ]:
nll_loss = output.loss
output_logits = output.logits[:, 256:-1]
device_id = output_logits.device
batch_size = output_logits.size(0)
output_gt = batch["labels"][:, 1:].to(device_id)
gripper_mask, item_mask, object_mask, target_mask, stage_mask, action_mask = action_tokenizer.get_mask(output_gt)

gripper_logits = output_logits[gripper_mask][:,action_tokenizer.action_token_begin_idx:action_tokenizer.tokenizer.vocab_size].view(batch_size,-1,action_tokenizer.n_bins)
item_logits = output_logits[item_mask][:,32003:32008]
object_logits = output_logits[object_mask][:,action_tokenizer.action_token_begin_idx:action_tokenizer.tokenizer.vocab_size].view(batch_size,-1,action_tokenizer.n_bins)
target_logits = output_logits[target_mask][:,action_tokenizer.action_token_begin_idx:action_tokenizer.tokenizer.vocab_size].view(batch_size,-1,action_tokenizer.n_bins)
stage_logits = output_logits[stage_mask][:,32012:32014]
action_logits = output_logits[action_mask][:,action_tokenizer.action_token_begin_idx:action_tokenizer.tokenizer.vocab_size].view(batch_size,-1,action_tokenizer.n_bins)

gt_gripper = batch['grippers'].to(device_id)
gt_item = batch['items'].to(device_id)
gt_object = batch['objects'].to(device_id)
gt_target = batch['targets'].to(device_id)
gt_stage = batch['stages'].to(device_id)
gt_action = batch['actions'].to(device_id)

#Gripper Loss
pred_gripper = action_tokenizer.decode(gripper_logits, soft = True)
assert pred_gripper.shape == gt_gripper.shape, f"Gripper shape {pred_gripper.shape} != {gt_gripper.shape}"
gripper_position_loss = F.mse_loss(pred_gripper[:,:3], gt_gripper[:,:3].to(torch.float32))
gripper_orientation_loss = action_tokenizer.angle_loss(pred_gripper[:,3:6], gt_gripper[:,3:6].to(torch.float32))
gripper_open_gt = gt_gripper[:,6].to(torch.int64)
gripper_open_loss = F.cross_entropy(gripper_logits[:,6,-2:], gripper_open_gt)

pred_object = action_tokenizer.decode(object_logits, soft = True)
assert pred_object.shape == gt_object.shape, f"Object shape {pred_object.shape} != {gt_object.shape}"
object_position_loss = F.mse_loss(pred_object[:,:3], gt_object[:,:3].to(torch.float32))

#Target Loss
pred_target = action_tokenizer.decode(target_logits, soft = True)
assert pred_target.shape == gt_target.shape, f"Target shape {pred_target.shape} != {gt_target.shape}"
target_position_loss = F.mse_loss(pred_target[:,:3], gt_target[:,:3].to(torch.float32))

#Stage Loss
gt_stage = gt_stage.clone().detach()
stage_loss = F.cross_entropy(stage_logits, gt_stage)

#Action Loss
pred_action = action_tokenizer.decode(action_logits, soft = True)
assert pred_action.shape == gt_action.shape, f"Action shape {pred_action.shape} != {gt_action.shape}"
action_position_loss = F.mse_loss(pred_action[:,:3], gt_action[:,:3].to(torch.float32))
action_orientation_loss = action_tokenizer.angle_loss(pred_action[:,3:6], gt_action[:,3:6].to(torch.float32))
action_open_gt = gt_action[:,6].to(torch.int64)
action_open_loss = F.cross_entropy(action_logits[:,6,-2:], action_open_gt)


: 

## Test

In [ ]:
step_idx, batch = next(enumerate(test_dataloader))

test_loss_dict = {
                "nll_loss": [],
                "gripper_position_loss": [],
                "gripper_orientation_loss": [],
                "gripper_open_loss": [],
                "item_loss": [],
                "object_position_loss": [],
                "target_position_loss": [],
                "stage_loss": [],
                "action_position_loss": [],
                "action_orientation_loss": [],
                "action_open_loss": [],
            }
with torch.no_grad(), torch.autocast("cuda", dtype=torch.bfloat16):
    output: CausalLMOutputWithPast = vla(
        input_ids=batch["input_ids"].to(device_id),
        attention_mask=batch["attention_mask"].to(device_id),
        pixel_values=batch["pixel_values"].to(torch.bfloat16).to(device_id),
        labels=batch["labels"],
    )
    output_start_idx = vla.vision_backbone.featurizer.patch_embed.num_patches
    loss_dict = action_tokenizer.get_loss(output, batch, output_start_idx)
    for k, v in loss_dict.items():
        test_loss_dict[k].append(v)

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/home/lawrence/anaconda3/lib/python3.11/site-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
